# QC-Py-04 - Research Workflow with QuantBook

> **De la recherche exploratoire à l'algorithme de production**
> Durée: 75 minutes | Niveau: Intermédiaire | Python + QuantConnect

---

## Objectifs d'Apprentissage

A la fin de ce notebook, vous serez capable de :

1. Maîtriser **QuantBook** pour la recherche exploratoire
2. Analyser des données avec **pandas** et **matplotlib**
3. Tester des stratégies en mode **recherche** (vectorized backtesting)
4. **Transitionner** du notebook vers un algorithme de production

## Prérequis

- Notebooks QC-Py-01, 02, 03 complétés
- Connaissance de pandas et matplotlib
- Compréhension du lifecycle QCAlgorithm

## Structure du Notebook

1. Introduction au workflow de recherche
2. Setup QuantBook
3. Exploration de données avec pandas
4. Test de stratégie en mode recherche
5. Utiliser shared/features.py
6. Transition Notebook → Algorithm
7. Exemple complet: Feature Engineering pour ML
8. Conclusion et prochaines étapes

---

## 1. Introduction au Workflow de Recherche (5 min)

### Workflow Recherche vs Backtest

QuantConnect propose deux modes de développement distincts :

| Aspect | Mode Recherche (QuantBook) | Mode Backtest (QCAlgorithm) |
|--------|---------------------------|-----------------------------|
| **Environment** | Jupyter Notebook | Event-driven algorithm |
| **Exécution** | Synchrone, interactive | Asynchrone, simulation temporelle |
| **API principale** | `QuantBook` | `QCAlgorithm` |
| **Lifecycle** | Pas de lifecycle | Initialize → OnData → OnEndOfAlgorithm |
| **Données** | `qb.History()` retourne DataFrame | `History()` retourne Slice |
| **Usage** | Exploration, analyse, prototypage | Backtesting, optimisation, production |
| **Visualisation** | matplotlib, seaborn directement | Via Charts API |

### QuantBook : API de Recherche Jupyter

**QuantBook** est une classe spécialisée qui permet d'interagir avec l'infrastructure QuantConnect dans un environnement Jupyter synchrone :

- Accès aux **mêmes données** que QCAlgorithm
- **Pas de lifecycle** : exécution cellule par cellule
- Retourne des **DataFrames pandas** au lieu de Slices
- Idéal pour **exploration rapide** et **prototypage**

### Différences Clés avec QCAlgorithm

```python
# QCAlgorithm (event-driven)
class MyAlgorithm(QCAlgorithm):
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.symbol = self.AddEquity("SPY").Symbol
    
    def OnData(self, data):
        if data.ContainsKey(self.symbol):
            # Event-driven logic
            pass

# QuantBook (synchronous)
qb = QuantBook()
spy = qb.AddEquity("SPY")
history = qb.History(qb.Securities.Keys, 252, Resolution.Daily)
df = history.loc["SPY"]  # Direct DataFrame access
```

**Workflow typique** :

1. **Recherche (QuantBook)** : Explorer données, tester hypothèses, visualiser
2. **Prototypage (QuantBook)** : Vectorized backtesting rapide
3. **Implémentation (QCAlgorithm)** : Traduire la logique en event-driven
4. **Validation (Backtest)** : Backtest complet avec réalisme
5. **Production (Live Trading)** : Déploiement

> **Note Pédagogique** : QuantBook est un outil de **recherche**, pas un outil de backtest. Les résultats peuvent différer légèrement d'un backtest complet QCAlgorithm (pas de slippage, frais simulés, etc.). Toujours valider avec un backtest avant production.

---

## 2. Setup QuantBook (10 min)

### Créer une Instance QuantBook

Commençons par importer les modules nécessaires et créer une instance de QuantBook.

In [ ]:
# Imports QuantConnect
from QuantConnect import *
from QuantConnect.Data import *
from QuantConnect.Research import *

# Imports pour analyse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Configuration matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("Imports réussis")

In [ ]:
# Créer instance QuantBook
qb = QuantBook()

print("QuantBook initialisé")
print(f"Type: {type(qb)}")

### Ajouter des Securities

Ajoutons SPY et AAPL pour notre analyse.

In [ ]:
# Ajouter securities
spy = qb.AddEquity("SPY", Resolution.Daily)
aapl = qb.AddEquity("AAPL", Resolution.Daily)

print(f"Securities ajoutées:")
print(f"  SPY: {spy.Symbol}")
print(f"  AAPL: {aapl.Symbol}")
print(f"\nTotal securities: {len(qb.Securities)}")

### Récupérer Historical Data

Récupérons 1 an de données (252 jours de trading).

In [ ]:
# Récupérer historical data
history = qb.History(qb.Securities.Keys, 252, Resolution.Daily)

print(f"Type de history: {type(history)}")
print(f"Shape: {history.shape}")
print(f"\nPremières lignes:")
history.head()

### Différences avec QCAlgorithm.History()

**Point clé** : `qb.History()` retourne directement un **DataFrame pandas** avec MultiIndex (symbol, time), contrairement à `QCAlgorithm.History()` qui retourne un objet `Slice`.

```python
# Dans QCAlgorithm
history = self.History([self.symbol], 252, Resolution.Daily)
# → Retourne Slice (itérable de TradeBar)

# Dans QuantBook
history = qb.History(qb.Securities.Keys, 252, Resolution.Daily)
# → Retourne directement pandas DataFrame
```

**Accès aux données d'un symbole spécifique** :

In [ ]:
# Extraire données SPY uniquement
df_spy = history.loc["SPY"]

print(f"SPY DataFrame shape: {df_spy.shape}")
print(f"\nColonnes: {df_spy.columns.tolist()}")
print(f"\nPremières lignes:")
df_spy.head()

> **Observation** : Le DataFrame contient les colonnes standard : `open`, `high`, `low`, `close`, `volume`. L'index est le timestamp.

---

## 3. Exploration de Données avec pandas (20 min)

### Statistiques Descriptives

Commençons par analyser les statistiques de base de SPY.

In [ ]:
# Statistiques descriptives
print("Statistiques SPY (1 an):")
print(df_spy[['close', 'volume']].describe())

### Calcul des Returns

Calculons les returns journaliers et cumulatifs.

In [ ]:
# Calculer returns journaliers
df_spy['returns'] = df_spy['close'].pct_change()

# Calculer returns cumulatifs
df_spy['cumulative_returns'] = (1 + df_spy['returns']).cumprod()

print("Returns ajoutés:")
print(df_spy[['close', 'returns', 'cumulative_returns']].tail())

### Visualisation des Returns Cumulatifs

In [ ]:
# Visualiser returns cumulatifs
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(df_spy.index, df_spy['close'], linewidth=2, color='navy')
plt.title('SPY - Prix de Clôture', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Prix ($)')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(df_spy.index, df_spy['cumulative_returns'], linewidth=2, color='darkgreen')
plt.axhline(y=1.0, color='red', linestyle='--', alpha=0.5, label='Baseline')
plt.title('SPY - Returns Cumulatifs', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Return Cumulatif')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistiques returns
total_return = df_spy['cumulative_returns'].iloc[-1] - 1
print(f"\nReturn total sur la période: {total_return*100:.2f}%")
print(f"Return annualisé: {(df_spy['returns'].mean() * 252)*100:.2f}%")
print(f"Volatilité annualisée: {(df_spy['returns'].std() * np.sqrt(252))*100:.2f}%")

### Volatilité Rolling

Calculons et visualisons la volatilité glissante sur 30 jours.

In [ ]:
# Calculer volatilité rolling 30 jours (annualisée)
df_spy['volatility_30d'] = df_spy['returns'].rolling(30).std() * np.sqrt(252)

# Visualisation
plt.figure(figsize=(14, 5))
plt.plot(df_spy.index, df_spy['volatility_30d'], linewidth=2, color='darkorange')
plt.title('SPY - Volatilité Rolling 30 Jours (Annualisée)', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Volatilité Annualisée')
plt.grid(True, alpha=0.3)
plt.axhline(y=df_spy['volatility_30d'].mean(), color='red', linestyle='--', alpha=0.5, 
            label=f'Moyenne: {df_spy["volatility_30d"].mean():.2%}')
plt.legend()
plt.tight_layout()
plt.show()

print(f"Volatilité moyenne: {df_spy['volatility_30d'].mean():.2%}")
print(f"Volatilité min: {df_spy['volatility_30d'].min():.2%}")
print(f"Volatilité max: {df_spy['volatility_30d'].max():.2%}")

> **Interprétation** : La volatilité rolling permet d'identifier les périodes de forte incertitude du marché (pics) et les périodes calmes (creux). Utile pour ajuster dynamiquement le sizing ou éviter de trader dans des conditions défavorables.

### Distribution des Returns

In [ ]:
# Visualiser distribution des returns
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df_spy['returns'].dropna(), bins=50, edgecolor='black', alpha=0.7, color='steelblue')
plt.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero Return')
plt.axvline(x=df_spy['returns'].mean(), color='green', linestyle='--', linewidth=2, 
            label=f'Mean: {df_spy["returns"].mean():.4f}')
plt.title('Distribution des Returns Journaliers', fontsize=14, fontweight='bold')
plt.xlabel('Return')
plt.ylabel('Fréquence')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
from scipy import stats
stats.probplot(df_spy['returns'].dropna(), dist="norm", plot=plt)
plt.title('Q-Q Plot (Normalité)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Test de normalité
stat, p_value = stats.shapiro(df_spy['returns'].dropna())
print(f"\nTest de Shapiro-Wilk (normalité):")
print(f"  Statistique: {stat:.4f}")
print(f"  P-value: {p_value:.4f}")
print(f"  Conclusion: {'Returns normaux' if p_value > 0.05 else 'Returns non-normaux (fat tails)'} (α=0.05)")

---

## 4. Test de Stratégie en Mode Recherche (20 min)

### Stratégie SMA Crossover

Testons une stratégie classique de croisement de moyennes mobiles : SMA 50 / SMA 200.

**Règles** :
- **Long** : SMA 50 > SMA 200 ("Golden Cross")
- **Flat/Short** : SMA 50 <= SMA 200 ("Death Cross")

### Calcul des Indicateurs

In [ ]:
# Calculer SMA 50 et 200
df_spy['sma_50'] = df_spy['close'].rolling(50).mean()
df_spy['sma_200'] = df_spy['close'].rolling(200).mean()

print("Indicateurs ajoutés:")
print(df_spy[['close', 'sma_50', 'sma_200']].tail())

### Génération des Signaux

Générons les signaux de trading basés sur le croisement.

In [ ]:
# Générer signaux
df_spy['signal'] = 0
df_spy.loc[df_spy['sma_50'] > df_spy['sma_200'], 'signal'] = 1   # Long
df_spy.loc[df_spy['sma_50'] <= df_spy['sma_200'], 'signal'] = -1  # Flat

# Position = signal décalé d'1 période (éviter lookahead bias)
df_spy['position'] = df_spy['signal'].shift(1)

print("Signaux et positions:")
print(df_spy[['close', 'sma_50', 'sma_200', 'signal', 'position']].tail(10))

> **Point Crucial** : Le `.shift(1)` sur la position est essentiel pour éviter le **lookahead bias**. On prend position au jour `t` en fonction du signal calculé au jour `t-1`.

### Backtesting Vectorisé

Calculons les returns de la stratégie de manière vectorisée.

In [ ]:
# Returns de la stratégie
df_spy['strategy_returns'] = df_spy['position'] * df_spy['returns']

# Returns cumulatifs de la stratégie
df_spy['strategy_cumulative'] = (1 + df_spy['strategy_returns']).cumprod()

print("Returns stratégie calculés:")
print(df_spy[['returns', 'strategy_returns', 'cumulative_returns', 'strategy_cumulative']].tail())

### Comparaison Visuelle avec Buy-and-Hold

In [ ]:
# Visualisation comparative
plt.figure(figsize=(14, 7))

# Subplot 1: Prix avec SMA
plt.subplot(2, 1, 1)
plt.plot(df_spy.index, df_spy['close'], label='SPY Close', linewidth=2, color='black', alpha=0.7)
plt.plot(df_spy.index, df_spy['sma_50'], label='SMA 50', linewidth=1.5, color='blue', alpha=0.7)
plt.plot(df_spy.index, df_spy['sma_200'], label='SMA 200', linewidth=1.5, color='red', alpha=0.7)
plt.title('SPY avec SMA 50/200', fontsize=14, fontweight='bold')
plt.ylabel('Prix ($)')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2: Returns cumulatifs
plt.subplot(2, 1, 2)
plt.plot(df_spy.index, df_spy['cumulative_returns'], label='Buy & Hold', 
         linewidth=2, color='gray', alpha=0.7)
plt.plot(df_spy.index, df_spy['strategy_cumulative'], label='SMA Crossover', 
         linewidth=2, color='darkgreen', alpha=0.9)
plt.axhline(y=1.0, color='red', linestyle='--', alpha=0.3)
plt.title('Comparaison Returns Cumulatifs', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Return Cumulatif')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Métriques de Performance

In [ ]:
# Calculer métriques

# Buy & Hold
bh_total_return = df_spy['cumulative_returns'].iloc[-1] - 1
bh_sharpe = (df_spy['returns'].mean() / df_spy['returns'].std()) * np.sqrt(252)

# Stratégie
strat_total_return = df_spy['strategy_cumulative'].iloc[-1] - 1
strat_sharpe = (df_spy['strategy_returns'].mean() / df_spy['strategy_returns'].std()) * np.sqrt(252)

# Max Drawdown (Buy & Hold)
cummax_bh = df_spy['cumulative_returns'].cummax()
drawdown_bh = (df_spy['cumulative_returns'] - cummax_bh) / cummax_bh
max_dd_bh = drawdown_bh.min()

# Max Drawdown (Stratégie)
cummax_strat = df_spy['strategy_cumulative'].cummax()
drawdown_strat = (df_spy['strategy_cumulative'] - cummax_strat) / cummax_strat
max_dd_strat = drawdown_strat.min()

# Affichage
print("="*60)
print("MÉTRIQUES DE PERFORMANCE")
print("="*60)
print(f"\n{'Métrique':<30} {'Buy & Hold':<15} {'SMA Crossover':<15}")
print("-"*60)
print(f"{'Return Total':<30} {bh_total_return:>13.2%} {strat_total_return:>13.2%}")
print(f"{'Sharpe Ratio':<30} {bh_sharpe:>13.2f} {strat_sharpe:>13.2f}")
print(f"{'Max Drawdown':<30} {max_dd_bh:>13.2%} {max_dd_strat:>13.2%}")
print(f"{'Volatilité Annualisée':<30} {df_spy['returns'].std() * np.sqrt(252):>13.2%} {df_spy['strategy_returns'].std() * np.sqrt(252):>13.2%}")
print("="*60)

# Nombre de trades
position_changes = (df_spy['position'].diff() != 0).sum()
print(f"\nNombre de changements de position: {position_changes}")

> **Analyse** :
> - Le **Sharpe Ratio** mesure le return ajusté au risque (plus élevé = meilleur)
> - Le **Max Drawdown** mesure la perte maximale depuis un pic (plus proche de 0 = meilleur)
> - Cette stratégie SMA est un **trend-following** : performant en tendance, mais génère des faux signaux en marché latéral
> 
> **Limitation** : Ce backtest vectorisé ne simule PAS les frais de transaction, le slippage, ou les contraintes de capital. Toujours valider avec un backtest complet QCAlgorithm.

---

## 5. Utiliser shared/features.py (10 min)

### Importer les Helpers

Le repository CoursIA fournit des helpers réutilisables dans `shared/`. Importons-les.

In [ ]:
# Ajouter shared/ au path
import sys
sys.path.append('../shared')

# Importer helpers
from features import calculate_returns, add_technical_features
from backtest_helpers import calculate_metrics, format_backtest_summary

print("Helpers importés avec succès")

### Calculer Returns Multi-Période

La fonction `calculate_returns` permet de calculer des returns sur plusieurs horizons temporels en une seule ligne.

In [ ]:
# Calculer returns sur 1, 5, 20 jours
returns_df = calculate_returns(df_spy['close'], periods=[1, 5, 20])

print("Returns multi-période:")
print(returns_df.tail())

# Merger avec df principal
df_spy = df_spy.join(returns_df, how='left')

### Ajouter Features Techniques

La fonction `add_technical_features` automatise l'ajout d'indicateurs techniques.

In [ ]:
# Ajouter features techniques
df_with_features = add_technical_features(
    df_spy, 
    indicators={
        'sma': [20, 50, 200],
        'rsi': 14,
        'macd': (12, 26, 9),
        'bb': (20, 2)
    }
)

print(f"Features ajoutées. Nouvelles colonnes: {df_with_features.shape[1] - df_spy.shape[1]}")
print(f"\nColonnes features:")
print([col for col in df_with_features.columns if col not in df_spy.columns])

### Calculer Métriques Standardisées

In [ ]:
# Simuler une equity curve (capital initial 100k)
initial_capital = 100000
equity_series = df_spy['strategy_cumulative'] * initial_capital
benchmark_series = df_spy['cumulative_returns'] * initial_capital

# Calculer métriques via helper
metrics = calculate_metrics(equity_series, benchmark=benchmark_series)

# Formatter résumé
summary = format_backtest_summary(metrics, strategy_name="SMA Crossover 50/200")

print(summary)

> **Avantage** : Les helpers `shared/` standardisent les calculs entre tous les notebooks QuantConnect. Réutilisation du code, moins d'erreurs, plus de cohérence.

---

## 6. Transition Notebook → Algorithm (15 min)

### Correspondances QuantBook → QCAlgorithm

Voici comment transformer le code testé en QuantBook vers un QCAlgorithm production-ready.

| Aspect | QuantBook (Recherche) | QCAlgorithm (Production) |
|--------|----------------------|-------------------------|
| **Setup** | `qb = QuantBook()` | `class MyAlgo(QCAlgorithm):` |
| **Initialisation** | Pas de méthode | `def Initialize(self):` |
| **Ajouter securities** | `qb.AddEquity("SPY")` | `self.AddEquity("SPY", Resolution.Daily)` |
| **Indicateurs** | Calcul pandas (`df['sma'] = df['close'].rolling(50).mean()`) | `self.SMA(symbol, 50)` (auto-warm-up) |
| **Historical data** | `qb.History()` → DataFrame | `self.History()` → Slice (event-driven) |
| **Trading logic** | Vectorisé (une fois) | Event-driven (`OnData`) |
| **Orders** | Simulation | `self.SetHoldings()`, `self.Liquidate()` |

### Code Testé en QuantBook (Récapitulatif)

```python
# Dans QuantBook (ci-dessus)
qb = QuantBook()
spy = qb.AddEquity("SPY")
history = qb.History(qb.Securities.Keys, 252, Resolution.Daily)
df_spy = history.loc["SPY"]

# Indicateurs
df_spy['sma_50'] = df_spy['close'].rolling(50).mean()
df_spy['sma_200'] = df_spy['close'].rolling(200).mean()

# Signaux
df_spy['signal'] = 0
df_spy.loc[df_spy['sma_50'] > df_spy['sma_200'], 'signal'] = 1
df_spy.loc[df_spy['sma_50'] <= df_spy['sma_200'], 'signal'] = -1

# Backtest vectorisé
df_spy['position'] = df_spy['signal'].shift(1)
df_spy['strategy_returns'] = df_spy['position'] * df_spy['returns']
```

### Transformation en QCAlgorithm

Voici le code équivalent pour un algorithme QuantConnect.

In [ ]:
# Code à copier dans un fichier .py QuantConnect

from AlgorithmImports import *

class SMACrossoverAlgorithm(QCAlgorithm):
    """
    Stratégie SMA Crossover 50/200 testée dans QC-Py-04-Research-Workflow.ipynb
    
    Règles:
    - Long quand SMA 50 > SMA 200
    - Flat quand SMA 50 <= SMA 200
    """
    
    def Initialize(self):
        # Configuration backtest
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Ajouter SPY
        self.symbol = self.AddEquity("SPY", Resolution.Daily).Symbol
        
        # Indicateurs (identiques au notebook)
        self.sma_50 = self.SMA(self.symbol, 50)
        self.sma_200 = self.SMA(self.symbol, 200)
        
        # Warm-up pour avoir les indicateurs prêts dès le début
        self.SetWarmup(200)
    
    def OnData(self, data):
        # Attendre que les indicateurs soient prêts
        if not self.sma_50.IsReady or not self.sma_200.IsReady:
            return
        
        # Logique identique au notebook (event-driven)
        if self.sma_50.Current.Value > self.sma_200.Current.Value:
            # Golden Cross → Long 100%
            if not self.Portfolio.Invested:
                self.SetHoldings(self.symbol, 1.0)
                self.Debug(f"{self.Time}: Golden Cross - Going Long")
        else:
            # Death Cross → Liquidate
            if self.Portfolio.Invested:
                self.Liquidate(self.symbol)
                self.Debug(f"{self.Time}: Death Cross - Liquidating")
    
    def OnEndOfAlgorithm(self):
        # Résumé final
        self.Debug(f"Final Portfolio Value: ${self.Portfolio.TotalPortfolioValue:,.2f}")

### Points Clés de la Transition

1. **Indicateurs** : `self.SMA()` remplace `df['sma'] = df['close'].rolling().mean()`
   - Auto-warm-up avec `SetWarmup(200)`
   - Vérifier `IsReady` avant utilisation

2. **Logique de trading** : De vectorisée à event-driven
   - `OnData()` appelé à chaque barre
   - Condition identique : `sma_50 > sma_200`
   - Pas de `.shift(1)` nécessaire (logique naturellement décalée)

3. **Orders** : `SetHoldings()` et `Liquidate()` au lieu de simulation pandas

4. **Backtest réaliste** : Frais de transaction, slippage, contraintes de capital simulés automatiquement

> **Workflow recommandé** :
> 1. Prototyper et tester la logique dans **QuantBook** (rapide, itératif)
> 2. Convertir en **QCAlgorithm** quand la logique est validée
> 3. Lancer un **backtest complet** pour validation finale
> 4. Optimiser les paramètres (notebook 15)
> 5. Déployer en **paper trading** (notebook 27)

---

## 7. Exemple Complet : Feature Engineering pour ML (10 min)

### Objectif

Préparer un dataset avec features pour le Machine Learning (preview du workflow du notebook QC-Py-18).

### Ajouter Features Techniques Complètes

In [ ]:
# Ajouter features complètes via helper
df_ml = add_technical_features(
    df_spy.copy(), 
    indicators={
        'sma': [10, 20, 50],
        'ema': [12, 26],
        'rsi': 14,
        'macd': (12, 26, 9)
    }
)

print(f"DataFrame ML shape: {df_ml.shape}")
print(f"\nFeatures disponibles:")
print(df_ml.columns.tolist())

### Créer Labels (Classification Binaire)

Créons des labels pour prédire la direction du mouvement à 5 jours.

In [ ]:
# Future returns (5 jours)
df_ml['future_returns_5d'] = df_ml['close'].pct_change(5).shift(-5)

# Labels binaires: 1 = hausse, 0 = baisse
df_ml['label'] = (df_ml['future_returns_5d'] > 0).astype(int)

print("Labels créés:")
print(df_ml[['close', 'future_returns_5d', 'label']].tail(10))

# Distribution des labels
print(f"\nDistribution labels:")
print(df_ml['label'].value_counts())
print(f"\nProportion hausse: {df_ml['label'].mean():.2%}")

### Nettoyage et Split Train/Test

In [ ]:
# Drop NaN (dus aux rolling et shift)
df_ml_clean = df_ml.dropna()

print(f"Dataset après nettoyage: {df_ml_clean.shape}")

# Split train/test (70/30, chronologique)
split_idx = int(len(df_ml_clean) * 0.7)
train_df = df_ml_clean.iloc[:split_idx]
test_df = df_ml_clean.iloc[split_idx:]

print(f"\nTrain size: {len(train_df)} ({len(train_df)/len(df_ml_clean):.1%})")
print(f"Test size: {len(test_df)} ({len(test_df)/len(df_ml_clean):.1%})")

# Distribution labels dans train/test
print(f"\nLabel distribution (train): {train_df['label'].value_counts().to_dict()}")
print(f"Label distribution (test): {test_df['label'].value_counts().to_dict()}")

### Feature Importance Preview

Visualisons rapidement l'importance des features avec un Random Forest simple.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Sélectionner features (indicateurs techniques uniquement)
feature_cols = [col for col in df_ml_clean.columns 
                if col.startswith(('sma_', 'ema_', 'rsi', 'macd', 'bb_'))]

X_train = train_df[feature_cols]
y_train = train_df['label']

# Entraîner Random Forest rapide
rf = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=42)
rf.fit(X_train, y_train)

# Feature importance
importances = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

# Visualisation
plt.figure(figsize=(10, 6))
plt.barh(importances['feature'], importances['importance'], color='steelblue')
plt.xlabel('Importance')
plt.title('Feature Importance (Random Forest)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("Top 5 features:")
print(importances.head())

### Sauvegarder pour ML (Optionnel)

Si vous utilisez LEAN CLI local, vous pouvez sauvegarder le dataset pour réutilisation.

In [ ]:
# Sauvegarder datasets (optionnel)
# train_df.to_csv('../data/SPY_features_train.csv')
# test_df.to_csv('../data/SPY_features_test.csv')

print("Dataset prêt pour notebooks ML (QC-Py-18 à QC-Py-21)")
print(f"\nFeatures: {len(feature_cols)}")
print(f"Samples: {len(df_ml_clean)}")
print(f"Train/Test split: {len(train_df)}/{len(test_df)}")

> **Lien avec QC-Py-18** : Ce workflow de feature engineering sera détaillé dans le notebook **QC-Py-18-ML-Features-Engineering** où nous couvrirons :
> - Feature engineering avancé (lag features, rolling stats, interactions)
> - Labeling strategies (classification, régression, multi-class)
> - Walk-forward validation
> - Feature selection et dimensionality reduction

---

## 8. Conclusion et Prochaines Étapes

### Récapitulatif

Dans ce notebook, nous avons appris à :

1. Utiliser **QuantBook** pour la recherche exploratoire synchrone
2. Analyser des données avec **pandas** (returns, volatilité, distributions)
3. Tester une stratégie SMA Crossover en mode **vectorized backtesting**
4. Utiliser les **helpers shared/** pour standardiser les analyses
5. **Transitionner** d'un prototype QuantBook vers un QCAlgorithm production
6. Préparer un **dataset ML** avec feature engineering

### Workflow Recherche → Production

```
1. QuantBook (Exploration)
   ↓
   - Analyser données
   - Tester hypothèses
   - Vectorized backtesting rapide
   ↓
2. QCAlgorithm (Implémentation)
   ↓
   - Convertir logique en event-driven
   - Backtest complet avec frais/slippage
   ↓
3. Optimisation (Notebook 15)
   ↓
   - Parameter optimization
   - Walk-forward validation
   ↓
4. Production (Notebook 27)
   ↓
   - Paper trading
   - Live trading
```

### Points Clés à Retenir

| Concept | Description |
|---------|-------------|
| **QuantBook** | API synchrone pour recherche Jupyter, retourne pandas DataFrame |
| **Vectorized Backtesting** | Rapide pour prototypage, mais ne simule pas réalisme (frais, slippage) |
| **Lookahead Bias** | Toujours utiliser `.shift(1)` sur les signaux pour éviter de "voir le futur" |
| **Transition** | QuantBook → QCAlgorithm = Vectorisé → Event-driven |
| **Helpers shared/** | Standardiser calculs entre notebooks pour cohérence |

### Limitations du Backtest Vectorisé

Le backtesting vectorisé en QuantBook est utile pour le prototypage rapide, mais a des limitations :

- Pas de simulation de **frais de transaction**
- Pas de simulation de **slippage**
- Pas de gestion du **capital** (assume toujours assez de cash)
- Pas de **warm-up** pour les indicateurs (calculs pandas directs)
- Risque de **lookahead bias** si `.shift()` oublié

**Toujours valider avec un backtest complet QCAlgorithm avant production.**

### Prochaines Étapes

#### Notebook Suivant : QC-Py-05-Universe-Selection

Dans le prochain notebook, nous apprendrons à :
- Gérer des **univers dynamiques** (coarse/fine selection)
- Filtrer par **dollar volume**, **fundamentals**
- Rebalancer automatiquement le portfolio
- Stratégies multi-assets

#### Exercice Suggéré

**Tester une stratégie RSI Mean Reversion en mode recherche** :

1. Calculer RSI 14 sur SPY
2. Règles :
   - Long quand RSI < 30 (oversold)
   - Short/Flat quand RSI > 70 (overbought)
3. Backtester en mode vectorisé
4. Comparer avec Buy & Hold
5. Convertir en QCAlgorithm

**Solution dans** : `algorithms/RSI_MeanReversion.py` (repository)

### Ressources Complémentaires

- [QuantConnect Research Documentation](https://www.quantconnect.com/docs/v2/research-environment)
- [QuantBook API Reference](https://www.quantconnect.com/docs/v2/research-environment/api-reference)
- [Pandas Time Series](https://pandas.pydata.org/docs/user_guide/timeseries.html)
- Helpers : `shared/features.py`, `shared/backtest_helpers.py`

---

**Notebook complété. Prêt pour QC-Py-05-Universe-Selection.**